# Unit 8: Advanced OOP & Design Principles

This unit builds on the object-oriented programming foundations from Unit 7. You will learn professional techniques for writing **robust**, **maintainable**, and **scalable** code—practices widely used in industry software development.

---

## Prerequisites

- Unit 7 (OOP Foundations) — classes, objects, inheritance basics

---

## Topics Covered

| # | Topic | Key Concept |
|---|-------|-------------|
| 1 | Encapsulation & Access Control | Protecting object state |
| 2 | Private Attributes & Name Mangling | Python's privacy conventions |
| 3 | Properties | Controlled attribute access |
| 4 | Abstract Base Classes | Defining contracts/interfaces |
| 5 | Class & Static Methods | Alternative constructors, utilities |
| 6 | SOLID Principles | Industry-standard design guidelines |
| 7 | Composition vs Inheritance | "Has-a" vs "Is-a" relationships |
| 8 | Factory Pattern | Centralized object creation |
| 9 | Singleton Pattern | Single instance guarantee |

## Learning Objectives

By the end of this unit, you will be able to:

**Core Skills**
- Apply encapsulation to protect object invariants
- Use properties to validate and control attribute access
- Define interfaces and contracts using Abstract Base Classes
- Use `@classmethod` and `@staticmethod` appropriately
- Choose between composition and inheritance using "has-a" vs "is-a" analysis

**Design Patterns**
- Implement the Factory pattern for flexible object creation
- Understand Singleton trade-offs and implement a thread-safe version

**Design Principles**
- Recognize and apply core SOLID principles
- Write code that is easier to test, extend, and maintain

---

# 1. Encapsulation & Access Control

## 1.1 What is Encapsulation?

**Encapsulation** is one of the four pillars of OOP (along with Abstraction, Inheritance, and Polymorphism).

> **Definition:** Encapsulation means bundling data (attributes) and methods that operate on that data within a single unit (class), while restricting direct access to some components.

### Benefits of Encapsulation

| Problem Without Encapsulation | Solution With Encapsulation |
|------------------------------|----------------------------|
| Anyone can set invalid values | Validation in setters/methods |
| Internal changes break external code | Stable public interface |
| Hard to track where data changes | All changes go through methods |
| Difficult to add logging/validation later | Central point of control |

### Python's Approach to Access Control

Python does **not** enforce access modifiers like Java/C# (`private`, `protected`, `public`).
Instead, it uses **naming conventions**:

| Convention | Meaning | Example |
|------------|---------|---------|
| `attr` | Public — part of the API | `self.name` |
| `_attr` | Protected/Internal — use within class or subclasses | `self._balance` |
| `__attr` | Private — name-mangled to prevent accidental access | `self.__secret` |

> **Note:** Python follows the principle "We're all consenting adults here." The language trusts developers to follow conventions rather than enforcing strict access control.

## 1.2 Example: Bank Account Without Protection

In [ ]:
# Without encapsulation - anyone can modify balance directly
class BankAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = balance  # Public attribute - no protection!

# Creating an account
acc = BankAccount("Alice", 100)
print(f"Owner: {acc.owner}")
print(f"Balance: ${acc.balance}")

# Problem: Nothing prevents invalid operations!
acc.balance = -999  # This should not be allowed!
print(f"After invalid change: ${acc.balance}")  # Oops!

## 1.3 Protected Attributes (Single Underscore)

The single underscore `_attr` signals that an attribute is intended for internal use. While not enforced by the interpreter, professional Python developers respect this convention.

In [ ]:
# With encapsulation - balance is protected and accessed via methods
class SafeAccount:
    """A bank account with protected balance and validation."""
    
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self._balance = float(balance)  # Protected: underscore convention

    def deposit(self, amount):
        """Add money to the account."""
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self._balance += amount
        return self._balance

    def withdraw(self, amount):
        """Remove money from the account."""
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if amount > self._balance:
            raise ValueError(f"Insufficient funds. Available: ${self._balance}")
        self._balance -= amount
        return self._balance

    def get_balance(self):
        """Return current balance (read-only access)."""
        return self._balance

# Usage
account = SafeAccount("Bob", 100)
print(f"Initial balance: ${account.get_balance()}")

account.deposit(50)
print(f"After deposit: ${account.get_balance()}")

account.withdraw(30)
print(f"After withdrawal: ${account.get_balance()}")

# Try invalid operations
try:
    account.deposit(-50)
except ValueError as e:
    print(f"Error: {e}")

try:
    account.withdraw(500)
except ValueError as e:
    print(f"Error: {e}")

---

# 2. Private Attributes & Name Mangling

## 2.1 Double Underscore Prefix: Name Mangling

When you use `__attr` (double underscore prefix), Python performs **name mangling**:
- `self.__balance` becomes `self._ClassName__balance`

This mechanism prevents **accidental** access and avoids name collisions in inheritance hierarchies.

> **Important:** This is not true privacy—it is designed to prevent accidents, not to secure data from determined access.

In [ ]:
# Demonstrating name mangling
class SecureAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.__balance = float(balance)  # Name-mangled attribute

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self.__balance += float(amount)

    def get_balance(self):
        return self.__balance

# Create account
acc = SecureAccount("Alice", 100)
acc.deposit(50)
print(f"Balance via method: ${acc.get_balance()}")

# Direct access fails
try:
    print(acc.__balance)  # AttributeError!
except AttributeError as e:
    print(f"Direct access blocked: {e}")

# But name-mangled access still works (NOT recommended!)
print(f"Mangled name access: ${acc._SecureAccount__balance}")

# Show all attributes
print(f"\nAll attributes: {[attr for attr in dir(acc) if 'balance' in attr.lower()]}")

## 2.2 Name Mangling in Inheritance

Name mangling prevents attribute name collisions between parent and child classes:

In [ ]:
# Name mangling prevents accidental overrides in inheritance
class Parent:
    def __init__(self):
        self.__secret = "Parent's secret"
    
    def reveal_parent_secret(self):
        return self.__secret

class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__secret = "Child's secret"  # Different attribute!
    
    def reveal_child_secret(self):
        return self.__secret

child = Child()
print(f"Parent's secret: {child.reveal_parent_secret()}")
print(f"Child's secret: {child.reveal_child_secret()}")

# Both secrets exist independently due to name mangling
print(f"\nAttributes: {[a for a in dir(child) if 'secret' in a]}")

## 2.3 Choosing the Right Convention

| Convention | Use Case | Example |
|------------|----------|---------|
| `attr` | Part of public API | `self.name`, `self.owner` |
| `_attr` | Internal implementation detail | `self._cache`, `self._balance` |
| `__attr` | Avoiding name clashes in inheritance | `self.__id` in base class |

> **Best Practice:** Start with `_attr` for internal attributes. Only use `__attr` when you specifically need to prevent inheritance conflicts.

---

# 3. Properties — Pythonic Attribute Access

## 3.1 The Problem with Getters and Setters

In languages like Java and C#, explicit getter/setter methods are common:
```java
public double getBalance() { return this.balance; }
public void setBalance(double value) { this.balance = value; }
```

This approach is verbose. Python provides a more elegant solution: **properties**.

## 3.2 What Are Properties?

Properties allow you to:
- Access attributes with **simple syntax** (`obj.attr`)
- Execute **validation logic** behind the scenes
- Create **read-only** attributes
- Compute **derived values** on-the-fly

Properties are defined using the `@property` decorator.

In [ ]:
# Temperature class with property validation
class Temperature:
    """Temperature with automatic validation and unit conversion."""
    
    ABSOLUTE_ZERO = -273.15  # Kelvin = 0
    
    def __init__(self, celsius):
        self._celsius = celsius  # Uses the setter!

    @property
    def celsius(self):
        """Get temperature in Celsius."""
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        """Set temperature in Celsius with validation."""
        value = float(value)
        if value < self.ABSOLUTE_ZERO:
            raise ValueError(f"Temperature cannot be below absolute zero ({self.ABSOLUTE_ZERO}°C)")
        self._celsius = value

    @property
    def fahrenheit(self):
        """Get temperature in Fahrenheit (computed property)."""
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        """Set temperature using Fahrenheit."""
        self._celsius = (float(value) - 32) * 5/9  # Converts and validates
    
    @property
    def kelvin(self):
        """Get temperature in Kelvin."""
        return self._celsius - self.ABSOLUTE_ZERO
    
    def __repr__(self):
        return f"Temperature({self._celsius}°C / {self.fahrenheit}°F / {self.kelvin}K)"

# Usage examples
t = Temperature(20)
print(f"Created: {t}")

print(f"Celsius: {t.celsius}°C")
print(f"Fahrenheit: {t.fahrenheit}°F")
print(f"Kelvin: {t.kelvin}K")


t.celsius = 25
print(f"After setting Celsius: {t}")

t.fahrenheit = 98.6  # Human body temperature
print(f"After setting Fahrenheit: {t}")

# Try invalid value
try:
    t.celsius = -300  # Below absolute zero
except ValueError as e:
    print(f"Validation error: {e}")

## 3.3 Read-Only Properties

Defining only a getter (no setter) makes an attribute **read-only**:

In [ ]:
# Read-only properties example
class User:
    """User with immutable username and computed email."""
    
    def __init__(self, username, domain="company.com"):
        self._username = username
        self._domain = domain
        self._created_at = "2024-01-15"  # Simulated timestamp

    @property
    def username(self):
        """Username is read-only after creation."""
        return self._username
    
    @property
    def email(self):
        """Email is computed from username and domain."""
        return f"{self._username.lower()}@{self._domain}"
    
    @property
    def created_at(self):
        """Creation timestamp is immutable."""
        return self._created_at

# Usage
user = User("AliceSmith")
print(f"Username: {user.username}")
print(f"Email: {user.email}")
print(f"Created: {user.created_at}")

# Try to modify read-only property
try:
    user.username = "BobJones"
except AttributeError as e:
    print(f"\nCannot modify: {e}")

## 3.4 Cashing with Properties

cashing (lazy loading)

In [ ]:
class CachedData:
    """Demonstrates property with cache clearing via deleter."""
    
    def __init__(self, raw_data):
        self._raw_data = raw_data
        self._squaredList = None  # Cache
    
    @property
    def raw_data(self):
        return self._raw_data

    @raw_data.setter
    def raw_data(self, newList):
        self._squaredList = None
        self._raw_data = newList
    
    @property
    def squaredList(self):
        """Lazy-computed processed data with caching."""
        if self._squaredList is None:
            print("Computing processed data...")
            self._squaredList = [x ** 2 for x in self._raw_data]  # Expensive operation
        return self._squaredList.copy()


# Usage
data = CachedData([1, 2, 3, 4, 5, 0,  -6, 7, 8, 9, 10])
print("First Call")
%time data.squaredList  # Computes

print("\nSecond Call")
%time data.squaredList  # Computes

print(f"\nSquared List: {data.squaredList}")

## 3.5 Property with Deleter

You can also define behavior for `del obj.attr`:

In [ ]:
# Complete property example with getter, setter, and deleter
class CachedData:
    """Demonstrates property with cache clearing via deleter."""
    
    def __init__(self, raw_data):
        self._raw_data = raw_data
        self._processed = None  # Cache
    
    @property
    def processed(self):
        """Lazy-computed processed data with caching."""
        if self._processed is None:
            print("Computing processed data...")
            self._processed = [x * 2 for x in self._raw_data]  # Expensive operation
        return self._processed
    
    @processed.deleter
    def processed(self):
        """Clear the cache."""
        print("Clearing cache...")
        self._processed = None

# Usage
data = CachedData([1, 2, 3, 4, 5])
%time data.processed  # Computes
%time data.processed  # Uses cache

print(f"\nProcessed Data: {data.processed}\n")

del data.processed  # Clear cache
%time print(data.processed)  # Recomputes

---

# 4. Abstract Base Classes (ABC)

## 4.1 Enforcing Interfaces

Consider building a system where different storage backends must implement `save()` and `load()`. How do you ensure all implementations follow this contract?

**Without ABCs:**
- No compile-time or import-time checks
- Errors only appear at runtime when methods are called
- Documentation is the only "contract"

**With ABCs:**
- Python raises `TypeError` if you try to instantiate a class without implementing all abstract methods
- Clear contract visible in code
- IDE support for detecting missing methods

## 4.2 Creating an Abstract Base Class

In [ ]:
from abc import ABC, abstractmethod

class Storage(ABC):
    """Abstract base class defining storage interface.
    
    Any class inheriting from Storage MUST implement save() and load().
    """
    
    @abstractmethod
    def save(self, data):
        """Save data to storage."""
        pass

    @abstractmethod
    def load(self):
        """Load and return data from storage."""
        pass
    
    # Non-abstract method - provides default implementation
    def exists(self):
        """Check if storage has data (can be overridden)."""
        try:
            return self.load() is not None
        except:
            return False

# Try to instantiate abstract class directly
try:
    s = Storage()
except TypeError as e:
    print(f"Cannot instantiate abstract class: {e}")

## 4.3 Implementing the Interface

The following concrete implementations satisfy the `Storage` contract:

In [ ]:
import json
import csv
from pathlib import Path

class JSONStorage(Storage):
    """Store data as JSON file."""
    
    def __init__(self, filepath):
        self.path = Path(filepath)

    def save(self, data):
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"Saved to {self.path}")

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", encoding="utf-8") as f:
            return json.load(f)


class CSVStorage(Storage):
    """Store data as CSV file."""
    
    def __init__(self, filepath, fieldnames):
        self.path = Path(filepath)
        self.fieldnames = list(fieldnames)

    def save(self, data):
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"Saved to {self.path}")

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", newline="", encoding="utf-8") as f:
            return list(csv.DictReader(f))


class MemoryStorage(Storage):
    """Store data in memory (useful for testing)."""
    
    def __init__(self):
        self._data = None
    
    def save(self, data):
        self._data = data
        print("Saved to memory")
    
    def load(self):
        return self._data


# Test all implementations
sample_data = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25}
]

print("=== JSON Storage ===")
json_store = JSONStorage("data_u8/people.json")
json_store.save(sample_data)
print(f"Loaded: {json_store.load()}\n")

print("=== CSV Storage ===")
csv_store = CSVStorage("data_u8/people.csv", ["name", "age"])
csv_store.save(sample_data)
print(f"Loaded: {csv_store.load()}\n")

print("=== Memory Storage ===")
mem_store = MemoryStorage()
mem_store.save(sample_data)
print(f"Loaded: {mem_store.load()}")

## 4.4 Programming to Interfaces

The real power of ABCs: code can depend on the **interface**, not specific implementations.

This enables:
- **Testability** — swap real storage with `MemoryStorage` for testing
- **Flexibility** — switch from JSON to a database without changing business logic
- **Clear contracts** — new developers know exactly what to implement

In [ ]:
# Function that works with ANY Storage implementation
def backup_and_verify(storage: Storage, data) -> bool:
    """Save data and verify it was stored correctly."""
    storage.save(data)
    loaded = storage.load()
    
    # Simple verification
    if len(loaded) != len(data):
        return False
    return True

# Works with any storage type!
print("JSON:", backup_and_verify(JSONStorage("data_u8/backup.json"), sample_data))
print("CSV:", backup_and_verify(CSVStorage("data_u8/backup.csv", ["name", "age"]), sample_data))
print("Memory:", backup_and_verify(MemoryStorage(), sample_data))

## 4.5 Abstract Properties

You can define abstract properties that subclasses must implement:

In [ ]:
# Abstract properties example
class Shape(ABC):
    """Abstract shape with required area and perimeter."""
    
    @property
    @abstractmethod
    def area(self):
        """Calculate area of the shape."""
        pass
    
    @property
    @abstractmethod
    def perimeter(self):
        """Calculate perimeter of the shape."""
        pass
    
    def describe(self):
        return f"Area: {self.area:.2f}, Perimeter: {self.perimeter:.2f}"


class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    @property
    def area(self):
        return self.width * self.height
    
    @property
    def perimeter(self):
        return 2 * (self.width + self.height)


class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def area(self):
        import math
        return math.pi * self.radius ** 2
    
    @property
    def perimeter(self):
        import math
        return 2 * math.pi * self.radius


# Usage
shapes = [Rectangle(4, 5), Circle(3)]
for shape in shapes:
    print(f"{shape.__class__.__name__}: {shape.describe()}")

---

# 5. Class Methods and Static Methods

## 5.1 Three Types of Methods

| Method Type | Decorator | First Parameter | Access |
|-------------|-----------|-----------------|--------|
| Instance method | (none) | `self` | Instance + class data |
| Class method | `@classmethod` | `cls` | Class data only |
| Static method | `@staticmethod` | (none) | No implicit access |

## 5.2 When to Use Each

- **Instance methods**: Most common. Require access to object state.
- **Class methods**: Alternative constructors, factory methods, accessing class-level data.
- **Static methods**: Utility functions logically grouped with the class.

In [ ]:
# Comprehensive example showing all three method types
class Person:
    """Person class demonstrating different method types."""
    
    # Class attribute - shared by all instances
    population = 0
    
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = int(birth_year)
        Person.population += 1
    
    # Instance method - operates on self
    def introduce(self):
        """Instance method: needs self."""
        return f"Hi, I'm {self.name}, born in {self.birth_year}"
    
    def age(self, current_year=2024):
        """Instance method: calculates age."""
        return current_year - self.birth_year
    
    # Class method - alternative constructor
    @classmethod
    def from_string(cls, text):
        """Class method: creates Person from 'name,year' string."""
        name, year = text.split(",")
        return cls(name.strip(), int(year.strip()))
    
    @classmethod
    def from_dict(cls, data):
        """Class method: creates Person from dictionary."""
        return cls(data["name"], data["birth_year"])
    
    @classmethod
    def get_population(cls):
        """Class method: access class-level data."""
        return cls.population
    
    # Static method - utility function
    @staticmethod
    def is_valid_year(year):
        """Static method: validation utility."""
        return 1900 <= int(year) <= 2024
    
    @staticmethod
    def calculate_age(birth_year, current_year=2024):
        """Static method: pure calculation, no instance needed."""
        return current_year - birth_year


# Instance method usage
p1 = Person("Alice", 1990)
print(p1.introduce())
print(f"Age: {p1.age()}")

# Class method usage - alternative constructors
p2 = Person.from_string("Bob, 1985")
p3 = Person.from_dict({"name": "Charlie", "birth_year": 2000})
print(f"\nCreated: {p2.name}, {p3.name}")
print(f"Population: {Person.get_population()}")

# Static method usage - no instance needed
print(f"\nIs 1990 valid? {Person.is_valid_year(1990)}")
print(f"Is 1800 valid? {Person.is_valid_year(1800)}")
print(f"Age for birth year 1990: {Person.calculate_age(1990)}")

## 5.3 Class Methods and Inheritance

Class methods work correctly with inheritance—`cls` refers to the actual class being called:

In [ ]:
# Class methods work correctly with inheritance
class Animal:
    def __init__(self, name):
        self.name = name
    
    @classmethod
    def create_baby(cls, parent_name):
        """Creates a baby of the same type."""
        return cls(f"Baby of {parent_name}")
    
    def speak(self):
        return "..."

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

# cls refers to the actual class, not Animal
dog_baby = Dog.create_baby("Rex")  # Creates a Dog, not Animal
cat_baby = Cat.create_baby("Whiskers")  # Creates a Cat, not Animal

print(f"{dog_baby.name} says: {dog_baby.speak()}")  # Dog's speak()
print(f"{cat_baby.name} says: {cat_baby.speak()}")  # Cat's speak()
print(f"Types: {type(dog_baby).__name__}, {type(cat_baby).__name__}")

---

# 6. SOLID Principles

## 6.1 Overview

**SOLID** is an acronym for five design principles that help create maintainable, flexible, and scalable object-oriented software:

| Letter | Principle | Key Idea |
|--------|-----------|----------|
| **S** | Single Responsibility | A class should have one reason to change |
| **O** | Open/Closed | Open for extension, closed for modification |
| **L** | Liskov Substitution | Subtypes must be substitutable for base types |
| **I** | Interface Segregation | Many specific interfaces over one general interface |
| **D** | Dependency Inversion | Depend on abstractions, not concretions |

The following sections demonstrate each principle with Python examples.

## 6.2 Single Responsibility Principle (SRP)

> "A class should have only one reason to change."

**Violation:** One class handles user data, validation, database operations, and email sending.

**Correct approach:** Separate classes for each responsibility.

In [ ]:
# ❌ BAD: One class doing too many things
class UserManagerBad:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def validate_email(self):  # Validation logic
        return "@" in self.email
    
    def save_to_database(self):  # Database logic
        print(f"Saving {self.name} to database...")
    
    def send_welcome_email(self):  # Email logic
        print(f"Sending welcome email to {self.email}...")


# ✅ GOOD: Each class has one responsibility
class User:
    """Data class - just holds user data."""
    def __init__(self, name, email):
        self.name = name
        self.email = email

class EmailValidator:
    """Validates email addresses."""
    @staticmethod
    def is_valid(email):
        return "@" in email and "." in email

class UserRepository:
    """Handles database operations for users."""
    def save(self, user):
        print(f"Saving {user.name} to database...")
    
    def find_by_email(self, email):
        print(f"Looking up user with email {email}...")

class EmailService:
    """Handles sending emails."""
    def send_welcome(self, user):
        print(f"Sending welcome email to {user.email}...")


# Usage with SRP
user = User("Alice", "alice@example.com")
if EmailValidator.is_valid(user.email):
    UserRepository().save(user)
    EmailService().send_welcome(user)

## 6.3 Open/Closed Principle (OCP)

> "Software entities should be open for extension, but closed for modification."

Add new functionality by creating new classes, not by modifying existing code.

In [ ]:
# ❌ BAD: Must modify existing code to add new discount types
class DiscountCalculatorBad:
    def calculate(self, price, discount_type):
        if discount_type == "percentage":
            return price * 0.9
        elif discount_type == "fixed":
            return price - 10
        elif discount_type == "seasonal":  # Had to modify class!
            return price * 0.8
        # Every new type requires modifying this class!


# ✅ GOOD: Open for extension, closed for modification
class Discount(ABC):
    @abstractmethod
    def apply(self, price):
        pass

class PercentageDiscount(Discount):
    def __init__(self, percent):
        self.percent = percent
    
    def apply(self, price):
        return price * (1 - self.percent / 100)

class FixedDiscount(Discount):
    def __init__(self, amount):
        self.amount = amount
    
    def apply(self, price):
        return max(0, price - self.amount)

class SeasonalDiscount(Discount):
    """New discount type - no modification to existing classes!"""
    def __init__(self, season_multiplier):
        self.multiplier = season_multiplier
    
    def apply(self, price):
        return price * self.multiplier

# Calculator doesn't need to change when adding new discount types
class DiscountCalculator:
    def calculate(self, price, discount: Discount):
        return discount.apply(price)


# Usage
calc = DiscountCalculator()
print(f"10% off $100: ${calc.calculate(100, PercentageDiscount(10))}")
print(f"$15 off $100: ${calc.calculate(100, FixedDiscount(15))}")
print(f"Seasonal 20% off: ${calc.calculate(100, SeasonalDiscount(0.8))}")

## 6.4 Dependency Inversion Principle (DIP)

> "Depend on abstractions, not concretions."

High-level modules should not depend on low-level modules. Both should depend on abstractions.

This is exactly what was demonstrated with the `Storage` ABC earlier.

In [ ]:
# ❌ BAD: High-level class depends on low-level implementation
class ReportGeneratorBad:
    def __init__(self):
        self.storage = JSONStorage("reports/report.json")  # Hard dependency!
    
    def generate(self, data):
        # Process data...
        self.storage.save(data)


# ✅ GOOD: Depend on abstraction (interface)
class ReportGenerator:
    def __init__(self, storage: Storage):  # Accepts any Storage!
        self.storage = storage
    
    def generate(self, data):
        # Process data...
        processed = {"report": data, "generated": "2024-01-15"}
        self.storage.save(processed)
        return processed


# Easy to swap implementations
report_data = [{"metric": "sales", "value": 100}]

# For production: use JSON
json_gen = ReportGenerator(JSONStorage("data_u8/report.json"))
json_gen.generate(report_data)

# For testing: use Memory (fast, no files)
mem_gen = ReportGenerator(MemoryStorage())
result = mem_gen.generate(report_data)
print(f"Generated: {result}")

---

# 7. Composition vs Inheritance

## 7.1 Choosing the Right Relationship

When designing relationships between classes, consider:

| Question | Answer | Relationship |
|----------|--------|--------------|
| Is A **a type of** B? | Yes | Inheritance (is-a) |
| Does A **have** or **use** B? | Yes | Composition (has-a) |

### Examples

| Relationship | Type | Reasoning |
|--------------|------|-----------|
| Dog → Animal | Inheritance | A dog IS an animal |
| Car → Engine | Composition | A car HAS an engine |
| Manager → Employee | Inheritance | A manager IS an employee |
| Company → Employee | Composition | A company HAS employees |

## 7.2 Prefer Composition Over Inheritance

> "Favor composition over inheritance" — *Design Patterns* (Gang of Four)

**Problems with Inheritance:**
- Tight coupling between parent and child
- Changes to parent can break children
- Deep hierarchies become hard to understand
- Behavior cannot change at runtime

**Benefits of Composition:**
- Loose coupling
- Components can be swapped at runtime
- Easier to test (mock components)
- More flexible combinations

In [ ]:
## 7.3 Composition Example: Report with Swappable Exporters

In [ ]:
# Composition: Report HAS an exporter (can be swapped)

class Exporter(ABC):
    """Interface for exporters."""
    @abstractmethod
    def export(self, title, data):
        pass

class TextExporter(Exporter):
    """Export as plain text."""
    def export(self, title, data):
        lines = [f"=== {title} ===", "-" * 30]
        for key, value in data.items():
            lines.append(f"  {key}: {value}")
        lines.append("-" * 30)
        return "\n".join(lines)

class JSONExporter(Exporter):
    """Export as JSON."""
    def export(self, title, data):
        return json.dumps({"title": title, "data": data}, indent=2)

class HTMLExporter(Exporter):
    """Export as HTML."""
    def export(self, title, data):
        rows = "".join(f"<tr><td>{k}</td><td>{v}</td></tr>" for k, v in data.items())
        return f"<h1>{title}</h1><table>{rows}</table>"


class Report:
    """Report uses composition - exporter can be changed at runtime."""
    
    def __init__(self, title, data, exporter: Exporter):
        self.title = title
        self.data = data
        self.exporter = exporter  # Composition: Report HAS an exporter
    
    def export(self):
        return self.exporter.export(self.title, self.data)
    
    def change_exporter(self, new_exporter: Exporter):
        """Change format at runtime - only possible with composition!"""
        self.exporter = new_exporter


# Create report with text exporter
report = Report("Sales Summary", {"revenue": 50000, "units": 1200}, TextExporter())
print(report.export())
print()

# Switch to JSON at runtime!
report.change_exporter(JSONExporter())
print(report.export())
print()

# Switch to HTML
report.change_exporter(HTMLExporter())
print(report.export())

## 7.4 When Inheritance Is Appropriate

Inheritance works well when:
- There is a genuine "is-a" relationship
- Subclasses truly specialize the parent
- The hierarchy is shallow (2–3 levels maximum)
- Polymorphism is needed (treating different types uniformly)

In [ ]:
# Good use of inheritance: Account types
class BankAccountBase:
    """Base account with common functionality."""
    
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self._balance = float(balance)

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self._balance += float(amount)
        return self._balance
    
    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        self._balance -= float(amount)
        return self._balance

    @property
    def balance(self):
        return self._balance
    
    def __repr__(self):
        return f"{self.__class__.__name__}(owner={self.owner}, balance={self._balance})"


class SavingsAccount(BankAccountBase):
    """Savings account: earns interest."""
    
    def __init__(self, owner, balance=0.0, interest_rate=0.02):
        super().__init__(owner, balance)
        self.interest_rate = interest_rate
    
    def apply_interest(self):
        """Apply monthly interest."""
        interest = self._balance * self.interest_rate
        self._balance += interest
        return interest


class CheckingAccount(BankAccountBase):
    """Checking account: has overdraft protection."""
    
    def __init__(self, owner, balance=0.0, overdraft_limit=100):
        super().__init__(owner, balance)
        self.overdraft_limit = overdraft_limit
    
    def withdraw(self, amount):
        """Override: allows overdraft up to limit."""
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self._balance + self.overdraft_limit:
            raise ValueError(f"Exceeds overdraft limit (max: ${self._balance + self.overdraft_limit})")
        self._balance -= float(amount)
        return self._balance


# Polymorphism: treat all accounts uniformly
accounts = [
    SavingsAccount("Alice", 1000, interest_rate=0.03),
    CheckingAccount("Bob", 500, overdraft_limit=200),
]

# Common operations work on all account types
for acc in accounts:
    acc.deposit(100)
    print(acc)

# Type-specific operations
savings = accounts[0]
interest_earned = savings.apply_interest()
print(f"\nInterest earned: ${interest_earned:.2f}")

checking = accounts[1]
checking.withdraw(650)  # Uses overdraft!
print(f"Checking after overdraft: {checking}")

---

# 8. Design Pattern: Factory

## 8.1 Overview

The **Factory** pattern centralizes object creation logic. Instead of calling constructors directly, you use a factory method or class.

### When to Use Factory

| Scenario | Benefit |
|----------|---------|
| Multiple similar classes (Developer, Manager, Intern) | Single creation point |
| Complex construction logic | Hides complexity |
| Creation depends on configuration or input | Flexible instantiation |
| Need to decouple code from specific classes | Easier maintenance |

## 8.2 Example: Employee Types

In [ ]:
# Employee class hierarchy
class Employee:
    """Base employee class."""
    
    def __init__(self, name, employee_id):
        self.name = name
        self.employee_id = employee_id

    def role(self):
        return "Employee"
    
    def annual_bonus(self):
        return 1000
    
    def __repr__(self):
        return f"{self.role()}(name={self.name}, id={self.employee_id})"


class Developer(Employee):
    def role(self):
        return "Developer"
    
    def annual_bonus(self):
        return 3000


class Manager(Employee):
    def __init__(self, name, employee_id, team_size=0):
        super().__init__(name, employee_id)
        self.team_size = team_size
    
    def role(self):
        return "Manager"
    
    def annual_bonus(self):
        return 5000 + (self.team_size * 500)


class Intern(Employee):
    def role(self):
        return "Intern"
    
    def annual_bonus(self):
        return 500

## 8.3 Factory with Registry Pattern

The registry pattern maps type names to classes, making the factory extensible:

In [ ]:
class EmployeeFactory:
    """Factory for creating employees with registry pattern."""
    
    # Registry maps type names to classes
    _registry = {
        "employee": Employee,
        "developer": Developer,
        "manager": Manager,
        "intern": Intern,
    }
    
    # Counter for auto-generating IDs
    _next_id = 1000

    @classmethod
    def create(cls, employee_type, name, **kwargs):
        """Create an employee by type name.
        
        Args:
            employee_type: String like "developer", "manager"
            name: Employee name
            **kwargs: Additional arguments for specific types
        """
        key = employee_type.lower().strip()
        
        if key not in cls._registry:
            available = ", ".join(cls._registry.keys())
            raise ValueError(f"Unknown type '{employee_type}'. Available: {available}")
        
        # Auto-generate ID
        employee_id = cls._next_id
        cls._next_id += 1
        
        # Create instance
        employee_class = cls._registry[key]
        return employee_class(name, employee_id, **kwargs)
    
    @classmethod
    def register(cls, type_name, employee_class):
        """Register a new employee type (extensibility!)."""
        cls._registry[type_name.lower()] = employee_class
    
    @classmethod
    def available_types(cls):
        """List available employee types."""
        return list(cls._registry.keys())


# Create employees using the factory
team = [
    EmployeeFactory.create("developer", "Alice Chen"),
    EmployeeFactory.create("developer", "Bob Smith"),
    EmployeeFactory.create("manager", "Carol White", team_size=5),
    EmployeeFactory.create("intern", "Dave Brown"),
]

print("Team members:")
for emp in team:
    print(f"  {emp} - Bonus: ${emp.annual_bonus()}")

print(f"\nAvailable types: {EmployeeFactory.available_types()}")

# Try invalid type
try:
    EmployeeFactory.create("ceo", "Big Boss")
except ValueError as e:
    print(f"\nError: {e}")

## 8.4 Extending the Factory

The registry pattern allows adding new types without modifying factory code:

In [ ]:
# Add new employee type without touching EmployeeFactory code!
class Contractor(Employee):
    """Contractor - temporary employee."""
    
    def __init__(self, name, employee_id, hourly_rate=50):
        super().__init__(name, employee_id)
        self.hourly_rate = hourly_rate
    
    def role(self):
        return "Contractor"
    
    def annual_bonus(self):
        return 0  # Contractors don't get bonuses

# Register the new type
EmployeeFactory.register("contractor", Contractor)

# Now we can create contractors!
contractor = EmployeeFactory.create("contractor", "Eve Wilson", hourly_rate=75)
print(f"New hire: {contractor}")
print(f"Hourly rate: ${contractor.hourly_rate}")
print(f"Available types: {EmployeeFactory.available_types()}")

---

# 9. Design Pattern: Singleton

## 9.1 Overview

The **Singleton** pattern ensures a class has only **one instance** and provides global access to it.

### Common Use Cases

| Use Case | Rationale |
|----------|-----------|
| Configuration/Settings | Single source of truth |
| Database connection pool | Shared resource |
| Logger | Consistent logging across application |
| Cache | Shared cache instance |

### Trade-offs

| Pros | Cons |
|------|------|
| Single instance guaranteed | Global state (harder to test) |
| Lazy initialization | Hidden dependencies |
| Global access point | Tight coupling |

## 9.2 Implementation Using `__new__`

In [ ]:
# Method 1: Classic Singleton using __new__
class ConfigSingleton:
    """Application configuration - only one instance exists."""
    
    _instance = None
    _initialized = False

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, settings=None):
        # Only initialize once!
        if ConfigSingleton._initialized:
            return
        
        self.settings = dict(settings or {})
        self._defaults = {
            "debug": False,
            "log_level": "INFO",
            "max_connections": 10,
        }
        ConfigSingleton._initialized = True
    
    def get(self, key, default=None):
        return self.settings.get(key, self._defaults.get(key, default))
    
    def set(self, key, value):
        self.settings[key] = value


# Test singleton behavior
config1 = ConfigSingleton({"env": "development", "debug": True})
config2 = ConfigSingleton({"env": "production"})  # Ignored! Already initialized

print(f"Same instance? {config1 is config2}")
print(f"Config env: {config1.get('env')}")  # First initialization wins
print(f"Config debug: {config2.get('debug')}")

# Changes are visible to all "instances"
config1.set("api_key", "secret123")
print(f"API key from config2: {config2.get('api_key')}")

## 9.3 Pythonic Alternative: Module-Level Singleton

In Python, **modules are singletons by nature**. A simpler approach is to use module-level instances:

In [ ]:
# config.py (imagine this is a separate file)
# ------------------------------------------
# SETTINGS = {
#     "env": "development",
#     "debug": True,
# }
#
# def get_setting(key, default=None):
#     return SETTINGS.get(key, default)
#
# def set_setting(key, value):
#     SETTINGS[key] = value
# ------------------------------------------
#
# Usage in other files:
# from config import SETTINGS, get_setting
#
# This is simpler and more Pythonic than a Singleton class!

# Simulating module-level singleton
class _Config:
    """Internal config class."""
    def __init__(self):
        self.data = {"env": "development", "debug": True}
    
    def get(self, key, default=None):
        return self.data.get(key, default)
    
    def set(self, key, value):
        self.data[key] = value

# Module-level instance (created once when module loads)
config = _Config()

# Usage
print(f"Module singleton - env: {config.get('env')}")
config.set("new_setting", "value")
print(f"Module singleton - new_setting: {config.get('new_setting')}")

---

# 10. Exercises

The following exercises reinforce the concepts covered in this unit, progressing from basic to advanced.

| Exercise | Topics | Difficulty |
|----------|--------|------------|
| 1 | Properties, Encapsulation | Basic |
| 2 | Abstract Base Classes | Basic |
| 3 | Factory Pattern | Intermediate |
| 4 | Composition | Intermediate |
| 5 | Complete System | Advanced |

## Exercise 1: Product with Validated Properties

Create a `Product` class with the following requirements:

- `name` property (read-only after creation)
- `price` property with validation (must be > 0)
- `quantity` property with validation (must be >= 0)
- `total_value` computed property (price × quantity)
- `discount(percent)` method that reduces price

Test your implementation with the demo code provided.

In [ ]:
# Exercise 1: Product with Properties - Starter Code

class Product:
    """Product with validated properties."""
    
    def __init__(self, name, price, quantity=0):
        self._name = name
        # TODO: Use properties for price and quantity
        self.price = price
        self.quantity = quantity
    
    @property
    def name(self):
        """Name is read-only."""
        return self._name
    
    @property
    def price(self):
        # TODO: Return internal _price
        pass
    
    @price.setter
    def price(self, value):
        # TODO: Validate price > 0, then set _price
        pass
    
    @property
    def quantity(self):
        # TODO: Return internal _quantity
        pass
    
    @quantity.setter
    def quantity(self, value):
        # TODO: Validate quantity >= 0, then set _quantity
        pass
    
    @property
    def total_value(self):
        """Computed: price × quantity."""
        # TODO: Return price * quantity
        pass
    
    def discount(self, percent):
        """Apply a percentage discount to price."""
        # TODO: Reduce price by percent%
        pass
    
    def __repr__(self):
        return f"Product({self.name}, ${self.price:.2f}, qty={self.quantity})"


# === Demo (uncomment after implementation) ===
# p = Product("Laptop", 999.99, 5)
# print(p)
# print(f"Total value: ${p.total_value:.2f}")
#
# p.discount(10)  # 10% off
# print(f"After 10% discount: {p}")
#
# # Test validation
# try:
#     p.price = -100  # Should raise ValueError
# except ValueError as e:
#     print(f"Validation works: {e}")

## Exercise 2: Notifier Interface with ABC

Create a notification system using Abstract Base Classes:

- `Notifier` ABC with abstract `send(recipient, message)` method
- `ConsoleNotifier` — prints to console
- `ListNotifier` — stores messages in a list (useful for testing)
- `welcome_user(notifier, username)` function

This exercise demonstrates the Dependency Inversion Principle.

In [ ]:
# Exercise 2: Notifier Interface - Starter Code

from abc import ABC, abstractmethod

class Notifier(ABC):
    """Abstract base class for notification systems."""
    
    @abstractmethod
    def send(self, recipient, message):
        """Send a message to a recipient."""
        pass


class ConsoleNotifier(Notifier):
    """Print notifications to console."""
    
    def send(self, recipient, message):
        # TODO: Print formatted message like "[To: recipient] message"
        pass


class ListNotifier(Notifier):
    """Store notifications in a list (for testing)."""
    
    def __init__(self):
        self.messages = []
    
    def send(self, recipient, message):
        # TODO: Append (recipient, message) tuple to self.messages
        pass


def welcome_user(notifier: Notifier, username: str):
    """Send welcome message using any notifier."""
    # TODO: Use notifier.send() to send "Welcome, {username}!"
    pass


# === Demo (uncomment after implementation) ===
# # Test ConsoleNotifier
# console = ConsoleNotifier()
# welcome_user(console, "Alice")
#
# # Test ListNotifier (no output, stores messages)
# test_notifier = ListNotifier()
# welcome_user(test_notifier, "Bob")
# welcome_user(test_notifier, "Charlie")
# print(f"Stored messages: {test_notifier.messages}")

## Exercise 3: Vehicle Factory

Create a factory for different vehicle types:

- `Vehicle` base class with `brand`, `model`, and `wheels` attributes
- Subclasses: `Car` (4 wheels), `Motorcycle` (2 wheels), `Truck` (6 wheels)
- `VehicleFactory` with registry pattern
- Factory should auto-generate IDs

**Bonus:** Add a `register()` method to allow adding new vehicle types.

In [ ]:
# Exercise 3: Vehicle Factory - Starter Code

class Vehicle:
    """Base vehicle class."""
    wheels = 0
    
    def __init__(self, brand, model, vehicle_id):
        self.brand = brand
        self.model = model
        self.vehicle_id = vehicle_id
    
    def describe(self):
        return f"{self.brand} {self.model} ({self.wheels} wheels)"
    
    def __repr__(self):
        return f"{self.__class__.__name__}(id={self.vehicle_id}, {self.describe()})"


class Car(Vehicle):
    # TODO: Set wheels = 4
    pass


class Motorcycle(Vehicle):
    # TODO: Set wheels = 2
    pass


class Truck(Vehicle):
    # TODO: Set wheels = 6
    pass


class VehicleFactory:
    """Factory for creating vehicles."""
    
    _registry = {
        # TODO: Map "car" -> Car, "motorcycle" -> Motorcycle, "truck" -> Truck
    }
    _next_id = 1
    
    @classmethod
    def create(cls, vehicle_type, brand, model):
        # TODO: 
        # 1. Normalize vehicle_type (lowercase, strip)
        # 2. Check if type exists in registry
        # 3. Get next ID and increment counter
        # 4. Create and return vehicle instance
        pass
    
    @classmethod
    def register(cls, type_name, vehicle_class):
        # TODO: Add new type to registry
        pass


# === Demo (uncomment after implementation) ===
# fleet = [
#     VehicleFactory.create("car", "Toyota", "Camry"),
#     VehicleFactory.create("car", "Honda", "Civic"),
#     VehicleFactory.create("motorcycle", "Harley", "Sportster"),
#     VehicleFactory.create("truck", "Ford", "F-150"),
# ]
#
# print("Fleet:")
# for vehicle in fleet:
#     print(f"  {vehicle}")

## Exercise 4: Document with Exporters (Composition)

Build a document system using composition:

- `Document` class with `title` and `content`
- `Exporter` ABC with `export(doc)` method
- Concrete exporters: `MarkdownExporter`, `HTMLExporter`, `PlainTextExporter`
- Document can change its exporter at runtime

This exercise demonstrates the principle "favor composition over inheritance."

In [ ]:
# Exercise 4: Document with Exporters - Starter Code

class DocumentExporter(ABC):
    """Abstract base for document exporters."""
    
    @abstractmethod
    def export(self, title, content):
        """Export document to string format."""
        pass


class PlainTextExporter(DocumentExporter):
    def export(self, title, content):
        # TODO: Return plain text format
        # Title
        # -----
        # Content
        pass


class MarkdownExporter(DocumentExporter):
    def export(self, title, content):
        # TODO: Return Markdown format
        # # Title
        # 
        # Content
        pass


class HTMLExporter(DocumentExporter):
    def export(self, title, content):
        # TODO: Return HTML format
        # <h1>Title</h1><p>Content</p>
        pass


class Document:
    """Document with swappable export format."""
    
    def __init__(self, title, content, exporter: DocumentExporter = None):
        self.title = title
        self.content = content
        self.exporter = exporter or PlainTextExporter()
    
    def export(self):
        # TODO: Use self.exporter to export
        pass
    
    def set_exporter(self, exporter: DocumentExporter):
        # TODO: Change the exporter
        pass


# === Demo (uncomment after implementation) ===
# doc = Document("My Report", "This is the report content.")
#
# print("=== Plain Text ===")
# print(doc.export())
#
# doc.set_exporter(MarkdownExporter())
# print("\n=== Markdown ===")
# print(doc.export())
#
# doc.set_exporter(HTMLExporter())
# print("\n=== HTML ===")
# print(doc.export())

## Exercise 5: Library Management System

Build a comprehensive library system combining multiple concepts:

**1. Book class** with properties:
- `isbn` (read-only)
- `title` (read-only)
- `available` (can be changed)

**2. Member class** with:
- `member_id`, `name`
- `borrowed_books` list
- `borrow(book)` and `return_book(book)` methods

**3. Library class** (composition):
- Uses a `Storage` implementation for persistence
- `add_book(book)`, `register_member(member)`
- `checkout(member_id, isbn)`, `return_book(member_id, isbn)`
- `get_available_books()`

**4. Optional:** Add a `LibraryFactory` for creating pre-configured libraries

This exercise integrates encapsulation, properties, ABCs, composition, and the factory pattern.

In [ ]:
# Exercise 5: Library Management System - Starter Code

class Book:
    """Book with encapsulated attributes."""
    
    def __init__(self, isbn, title, author):
        self._isbn = isbn
        self._title = title
        self._author = author
        self._available = True
    
    @property
    def isbn(self):
        # TODO: Return ISBN (read-only)
        pass
    
    @property
    def title(self):
        # TODO: Return title (read-only)
        pass
    
    @property
    def author(self):
        # TODO: Return author (read-only)
        pass
    
    @property
    def available(self):
        # TODO: Return availability
        pass
    
    @available.setter
    def available(self, value):
        # TODO: Set availability (boolean)
        pass
    
    def __repr__(self):
        status = "Available" if self._available else "Checked Out"
        return f"Book('{self._title}' by {self._author}, {status})"


class Member:
    """Library member who can borrow books."""
    
    def __init__(self, member_id, name):
        self.member_id = member_id
        self.name = name
        self._borrowed_books = []
    
    @property
    def borrowed_books(self):
        """Return copy of borrowed books list."""
        return list(self._borrowed_books)
    
    def borrow(self, book: Book):
        # TODO: Add book to borrowed list if available
        # Mark book as unavailable
        # Raise ValueError if book not available
        pass
    
    def return_book(self, book: Book):
        # TODO: Remove book from borrowed list
        # Mark book as available
        # Raise ValueError if book not in borrowed list
        pass
    
    def __repr__(self):
        return f"Member({self.member_id}, {self.name}, {len(self._borrowed_books)} books)"


class Library:
    """Library using composition for storage."""
    
    def __init__(self, name, storage: Storage = None):
        self.name = name
        self._books = {}      # isbn -> Book
        self._members = {}    # member_id -> Member
        self._storage = storage or MemoryStorage()
    
    def add_book(self, book: Book):
        # TODO: Add book to library
        pass
    
    def register_member(self, member: Member):
        # TODO: Register member
        pass
    
    def checkout(self, member_id, isbn):
        # TODO: 
        # 1. Find member and book
        # 2. Call member.borrow(book)
        # 3. Return success message
        pass
    
    def return_book(self, member_id, isbn):
        # TODO:
        # 1. Find member and book
        # 2. Call member.return_book(book)
        # 3. Return success message
        pass
    
    def get_available_books(self):
        # TODO: Return list of available books
        pass
    
    def save_state(self):
        # TODO: Save library state using self._storage
        pass


# === Demo (uncomment after implementation) ===
# # Create library
# library = Library("City Library")
#
# # Add books
# library.add_book(Book("978-0-13-468599-1", "The Pragmatic Programmer", "Hunt & Thomas"))
# library.add_book(Book("978-0-596-51774-8", "JavaScript: The Good Parts", "Douglas Crockford"))
# library.add_book(Book("978-1-49-195017-1", "Fluent Python", "Luciano Ramalho"))
#
# # Register members
# library.register_member(Member("M001", "Alice"))
# library.register_member(Member("M002", "Bob"))
#
# # Checkout and return
# print(library.checkout("M001", "978-0-13-468599-1"))
# print(f"Available books: {library.get_available_books()}")
#
# print(library.return_book("M001", "978-0-13-468599-1"))
# print(f"Available books: {library.get_available_books()}")

---

## Summary

This unit covered the following key concepts:

| Topic | Key Takeaway |
|-------|--------------|
| Encapsulation | Use `_attr` and `__attr` conventions to protect internal state |
| Properties | Provide controlled, Pythonic attribute access |
| Abstract Base Classes | Define contracts that implementations must follow |
| `@classmethod` vs `@staticmethod` | Choose based on what data the method needs to access |
| Composition vs Inheritance | Prefer composition for flexibility; use inheritance for true "is-a" relationships |
| Factory Pattern | Centralize object creation using a registry |
| Singleton Pattern | Ensure single instance; consider module-level alternatives |

---

**Next:** Unit 9 — Computational Thinking & Algorithms